In [1]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas as pd
import numpy as np
import altair as alt
import project_functions2 as pf

from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

In [2]:
alt.renderers.enable('default')
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [3]:
stock_list = ['AMZN', 'AAPL', 'FB','GOOGL', 'MSFT', 'TSLA']
#stock_list = ['AMZN', 'AAPL', 'FB','GOOG', 'TSLA']
stock_objects = {}
for stock in stock_list:
    stock_objects[stock] = yf.Ticker(stock)

In [4]:
def neural_net_sklearn(combine_df, split_time, stock_dfs):
    scaler = MinMaxScaler()
    X = combine_df.iloc[:,:-1]
    y = combine_df.iloc[:,-1:]
    #X = stock_df['Days From IPO'].values.reshape(-1, 1)
    #y = stock_df['Close'].values.reshape(-1, 1)
    
    # Does train/Test Split on last year
    # Change the -50 to a differnt value to change split point
    split_mark = int(len(combine_df)-(split_time*len(stock_dfs)))
    X_train = X.head(split_mark)
    X_test = X.tail(len(combine_df) - split_mark)
    y_train = y.head(split_mark)
    y_test = y.tail(len(combine_df) - split_mark)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    stock_nn = MLPRegressor(hidden_layer_sizes = [500, 500, 500], random_state=45).fit(X_train_scaled, y_train)
    
    stock_nn_pred = stock_nn.predict(X_test_scaled)
    stock_nn_train_pred = stock_nn.predict(X_train_scaled)
    
    sell_line = stock_nn_pred*1.1
    buy_line = stock_nn_pred*0.9
    
    # Plots Results

    train_score = r2_score(y_train, stock_nn_train_pred)
    print("Training R2 Score: " + str(train_score))
    print()

    columns = []
    for key in stock_dfs:
        columns.append(key + ' Price')
        columns.append(key + ' Prediction')
    
    single_split_mark = int(len(stock_dfs[list(stock_dfs.keys())[0]])-split_time)
    results_df = pd.DataFrame(columns=columns, 
                              index=stock_dfs[list(stock_dfs.keys())[0]].tail(len(stock_dfs[list(stock_dfs.keys())[0]])-single_split_mark).index)

    i = 0
    for key in stock_dfs:
        single_split_mark = int(len(stock_dfs[key])-split_time)
        results_df[columns[i]] = stock_dfs[key].iloc[:,-1:].tail(len(stock_dfs[key]) - single_split_mark)
        curr_X_train = scaler.transform(stock_dfs[key].iloc[:,:-1].tail(len(stock_dfs[key]) - single_split_mark))
        results_df[columns[i+1]] = stock_nn.predict(curr_X_train)
        i += 2
        
    results_df.reset_index(inplace=True)
    results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
    line_plot = alt.Chart(results_df).mark_line().encode(
        x = 'Date',
        y = 'Price',
        color = 'Company'
    )
    
    model_score = r2_score(y_test.dropna(), stock_nn_pred[:len(y_test.dropna())])
    print("R2 Score: " + str(model_score))
    
    return line_plot

In [5]:
def neural_net_keras(combine_df, split_time, stock_dfs):
    scaler = MinMaxScaler()
    X = combine_df.iloc[:,:-1]
    y = combine_df.iloc[:,-1:]
    #X = stock_df['Days From IPO'].values.reshape(-1, 1)
    #y = stock_df['Close'].values.reshape(-1, 1)
    
    # Does train/Test Split on last year
    # Change the -50 to a differnt value to change split point
    split_mark = int(len(combine_df)-(split_time*len(stock_dfs)))
    X_train = X.head(split_mark)
    X_test = X.tail(len(combine_df) - split_mark)
    y_train = y.head(split_mark)
    y_test = y.tail(len(combine_df) - split_mark)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    stock_nn = Sequential()
    stock_nn.add(Dense(units=500, input_dim=X_train_scaled.shape[1], activation='relu'))
    stock_nn.add(Dense(units=500, activation='relu'))
    stock_nn.add(Dense(units=500, activation='relu'))
    stock_nn.compile(optimizer = 'adam', loss = 'mean_squared_error')
    stock_nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)
    
    stock_nn_pred = stock_nn.predict(X_test_scaled)
    stock_nn_train_pred = stock_nn.predict(X_train_scaled)
    
    sell_line = stock_nn_pred*1.1
    buy_line = stock_nn_pred*0.9
    
    # Plots Results

    train_score = r2_score(y_train, stock_nn_train_pred)
    print("Training R2 Score: " + str(train_score))
    print()

    columns = []
    for key in stock_dfs:
        columns.append(key + ' Price')
        columns.append(key + ' Prediction')
    
    single_split_mark = int(len(stock_dfs[list(stock_dfs.keys())[0]])-split_time)
    results_df = pd.DataFrame(columns=columns, 
                              index=stock_dfs[list(stock_dfs.keys())[0]].tail(len(stock_dfs[list(stock_dfs.keys())[0]])-single_split_mark).index)

    i = 0
    for key in stock_dfs:
        single_split_mark = int(len(stock_dfs[key])-split_time)
        results_df[columns[i]] = stock_dfs[key].iloc[:,-1:].tail(len(stock_dfs[key]) - single_split_mark)
        curr_X_train = scaler.transform(stock_dfs[key].iloc[:,:-1].tail(len(stock_dfs[key]) - single_split_mark))
        results_df[columns[i+1]] = stock_nn.predict(curr_X_train)
        i += 2
        
    results_df.reset_index(inplace=True)
    results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
    line_plot = alt.Chart(results_df).mark_line().encode(
        x = 'Date',
        y = 'Price',
        color = 'Company'
    )
    
    model_score = r2_score(y_test.dropna(), stock_nn_pred[:len(y_test.dropna())])
    print("R2 Score: " + str(model_score))
    
    return line_plot